In [1]:
!pip install -q keras
!pip install -q tensorflow


# Setup do dataset

### Clonando o dataset


In [2]:
import os
import shutil
from zipfile import ZipFile
from urllib.request import urlopen

if os.path.exists('dataset'):
  shutil.rmtree('dataset')

if os.path.exists('sample_data'):
  shutil.rmtree('sample_data')

if not os.path.isfile('dataset.zip'):
  zipresp = urlopen('https://github.com/ijda3/cnn-cavalo-galinha/raw/main/dataset.zip')
  tempzip = open("dataset.zip", "wb")
  tempzip.write(zipresp.read())
  tempzip.close()

zf = ZipFile("dataset.zip")
zf.extractall(path = './')
zf.close()

if os.path.exists('__MACOSX'):
  shutil.rmtree('__MACOSX')

### Separando arquivos de treino e teste

In [3]:
import os
from sklearn.model_selection import train_test_split

dataset_basedir = 'dataset'

if os.path.exists('training_set'):
  shutil.rmtree('training_set')

if os.path.exists('test_set'):
  shutil.rmtree('test_set')

def copy_files(files, dest, dir):
  for file in files:
    if not os.path.exists(dest):
      os.mkdir(dest)
      
    if not os.path.exists(dest + '/' + dir):
      os.mkdir(dest + '/' + dir)
      
    shutil.copy(dataset_basedir + '/' + dir + '/' + file, dest + '/' + dir + '/' + file)

for dir in ['cavalo', 'galinha']:
  files = os.listdir(dataset_basedir + '/' + dir)

  train, valid = train_test_split(files, train_size=0.8)

  copy_files(train, 'training_set', dir)
  copy_files(valid, 'test_set', dir)

#Construção da CNN

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import numpy as np
from keras.preprocessing import image

In [5]:
classificador = Sequential()
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))

classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))

classificador.add(Flatten())

classificador.add(Dense(units = 256, activation = 'relu'))
classificador.add(Dropout(0.1))
classificador.add(Dense(units = 256, activation = 'sigmoid'))
classificador.add(Dropout(0.1))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

#Treinamento da Rede

In [10]:
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)
gerador_teste = ImageDataGenerator(rescale = 1./255)

base_treinamento = gerador_treinamento.flow_from_directory('training_set',
                                                           target_size = (64, 64),
                                                           batch_size = 32,
                                                           class_mode = 'binary')
base_teste = gerador_teste.flow_from_directory('test_set',
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary')
classificador.fit(base_treinamento, steps_per_epoch = base_treinamento.samples / 32,
                            epochs = 10, validation_data = base_teste,
                            validation_steps = base_teste.samples / 32)


Found 4567 images belonging to 2 classes.
Found 1144 images belonging to 2 classes.
Epoch 1/10
142/142 [==============================] - 16s 114ms/step - loss: 0.0664 - accuracy: 0.9739 - val_loss: 1.1497 - val_accuracy: 0.7351
Epoch 2/10
142/142 [==============================] - 16s 113ms/step - loss: 0.0779 - accuracy: 0.9698 - val_loss: 0.3449 - val_accuracy: 0.8890
Epoch 3/10
142/142 [==============================] - 16s 111ms/step - loss: 0.0809 - accuracy: 0.9674 - val_loss: 0.2542 - val_accuracy: 0.9073
Epoch 4/10
142/142 [==============================] - 16s 112ms/step - loss: 0.0588 - accuracy: 0.9759 - val_loss: 0.2125 - val_accuracy: 0.9344
Epoch 5/10
142/142 [==============================] - 16s 113ms/step - loss: 0.0555 - accuracy: 0.9755 - val_loss: 0.5030 - val_accuracy: 0.8698
Epoch 6/10
142/142 [==============================] - 16s 112ms/step - loss: 0.0508 - accuracy: 0.9825 - val_loss: 0.3320 - val_accuracy: 0.9003
Epoch 7/10
142/142 [==========================

# Validação

### Separação do dataset usando o modelo treinado

In [11]:
import os
import shutil

if os.path.exists('dataset_result'):
  shutil.rmtree('dataset_result')

os.mkdir('dataset_result')
os.mkdir('dataset_result/cavalo')
os.mkdir('dataset_result/galinha')

images = []

root = 'dataset'

for path, subdirs, files in os.walk(root):
    for name in files:
        images.append(os.path.join(path, name))

for file in images:
  source = file
  file = os.path.basename(source)

  if file[0] == '.':
    continue

  imagem_teste = image.load_img(source, target_size = (64,64))

  imagem_teste = image.img_to_array(imagem_teste)
  imagem_teste/=255
  imagem_teste = np.expand_dims(imagem_teste, axis=0)

  previsao = classificador.predict(imagem_teste)

  if previsao > 0.5:
    dest = 'dataset_result/galinha/' + file
  else:
    dest = 'dataset_result/cavalo/' + file

  shutil.copy(source, dest)


### Computando o resultado

In [12]:
import os
import shutil
import pandas as pd

files_by_category = {
    'galinha': [],
    'cavalo': []
}

stats = {
    'galinha': {
        'total': 0,
        'acertos': 0,
        'erros': 0,
        'acuracia': 0
    },
    'cavalo': {
        'total': 0,
        'acertos': 0,
        'erros': 0,
        'acuracia': 0
    }
}


if os.path.exists('dataset_result_errado'):
  shutil.rmtree('dataset_result_errado')

os.mkdir('dataset_result_errado')
os.mkdir('dataset_result_errado/galinha')
os.mkdir('dataset_result_errado/cavalo')

for path, subdirs, files in os.walk('dataset'):
    for name in files:
        if path.find('galinha') >= 0:
            files_by_category['galinha'].append(name)
        elif path.find('cavalo') >= 0:
            files_by_category['cavalo'].append(name)

stats['galinha']['total'] = len(files_by_category['galinha'])
stats['cavalo']['total'] = len(files_by_category['cavalo'])

for path, subdirs, files in os.walk('dataset_result/'):
    for name in files:
        file = os.path.join(path, name)

        if path.find('galinha') >= 0:
            if name in files_by_category['galinha']:
                stats['galinha']['acertos'] += 1
            else:
                stats['cavalo']['erros'] += 1
                shutil.copy(
                    file, 'dataset_result_errado/galinha/' + name)

        elif path.find('cavalo') >= 0:
            if name in files_by_category['cavalo']:
                stats['cavalo']['acertos'] += 1
            else:
                stats['galinha']['erros'] += 1
                shutil.copy(
                    file, 'dataset_result_errado/cavalo/' + name)


stats['galinha']['acuracia'] = round(
    stats['galinha']['acertos'] / stats['galinha']['total'], 4)
stats['cavalo']['acuracia'] = round(
    stats['cavalo']['acertos'] / stats['cavalo']['total'], 4)

print(pd.DataFrame(stats).T)


          total  acertos  erros  acuracia
galinha  3096.0   3001.0   94.0    0.9693
cavalo   2617.0   2511.0  105.0    0.9595


In [13]:
shutil.make_archive('dataset_result_errado', 'zip', 'dataset_result_errado')

'/content/dataset_result_errado.zip'